In [1]:
import pandas as pd
hits = pd.read_csv("data/final_hits_10.csv")

#hits.head(5)
#hits.info()
#hits.describe()

In [2]:
import pandas as pd
products = pd.read_csv("data/final_products_10.csv")

# products.head(5)
# products.info()

In [3]:
import pandas as pd

sessions = pd.read_csv("data/final_sessions_10.csv")

# sessions.head(5)
# sessions.info()

# Data Import & Cleaning

### Converting to the standard currency in USA

Перевіряємо максимум перед діленям

In [4]:
print('Max of transactionRevenue:', hits["transactionRevenue"].max())
print('Max of transactionTax:', hits["transactionTax"].max())
print('Max of transactionShipping:', hits["transactionShipping"].max())
print('Max of productRevenue:', products["productRevenue"].max())
print('Max of productPrice:', products["productPrice"].max())

Max of transactionRevenue: 7003500000.0
Max of transactionTax: 691890000.0
Max of transactionShipping: 1138740000.0
Max of productRevenue: 7003500000.0
Max of productPrice: 250000000


In [5]:
hits["transactionRevenue_converted"] = hits["transactionRevenue"] / 1000000
hits["transactionTax_converted"] = hits["transactionTax"] / 1000000
hits["transactionShipping_converted"] = hits["transactionShipping"] / 1000000
products["productRevenue_converted"] = products["productRevenue"] / 1000000
products["productPrice_converted"] = products["productPrice"] / 1000000

Тепер перевіряємо, чи коректно все поділилося

In [6]:
print('Max of transactionRevenue:', hits["transactionRevenue_converted"].max())
print('Max of transactionTax:', hits["transactionTax_converted"].max())
print('Max of transactionShipping:', hits["transactionShipping_converted"].max())
print('Max of productRevenue:', products["productRevenue_converted"].max())
print('Max of productPrice:', products["productPrice_converted"].max())

Max of transactionRevenue: 7003.5
Max of transactionTax: 691.89
Max of transactionShipping: 1138.74
Max of productRevenue: 7003.5
Max of productPrice: 250.0


In [7]:
import numpy as np

placeholders = ['(not set)', '', '(none)', '(not provided)', 'not available in demo dataset']

def standardize_missing_values(df):
    return df.replace(placeholders, np.nan)

hits = standardize_missing_values(hits)
products = standardize_missing_values(products)
sessions = standardize_missing_values(sessions)

print("Missing values in final_hits_10:\n", hits.isnull().sum())
print("Missing values in final_products_10:\n", products.isnull().sum())
print("Missing values in final_sessions_10:\n", sessions.isnull().sum())

/var/folders/5r/4d3mqngx3c3bynq7lh4_h3tr0000gn/T/ipykernel_42629/2415537560.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return df.replace(placeholders, np.nan)


Missing values in final_hits_10:
 uniqueVisitId                         0
hitId                                 0
visitStartTime                        0
hitNumber                             0
time                                  0
hour                                  0
minute                                0
isEntrance                        98395
isExit                            98395
referer                           92695
pagePath                              0
hostname                              0
pageTitle                             9
eventCategory                    104750
eventAction                      104750
eventLabel                       114313
ecommerce_action_type                 0
ecommerce_step                        0
ecommerce_option_value           122841
transactionid                    124866
transactionRevenue               125167
transactionTax                   125239
transactionShipping              125165
transactionCurrencyCode           76396
group_

### Handle Geographic Data Completeness Variations

In [8]:
print(sessions[['continent', 'subcontinent', 'country', 'city']].isnull().sum())
print()
sessions['clean_continent'] = sessions['continent'].fillna('No data')
sessions['clean_subcontinent'] = sessions['subcontinent'].fillna('No data')
sessions['clean_country'] = sessions['country'].fillna('No data')
sessions['clean_city'] = sessions['city'].fillna('No data')
print("Перевіряємо, чи все коректно замінилося")
print(sessions[['clean_continent', 'clean_subcontinent', 'clean_country', 'clean_city']].isnull().sum())

continent          50
subcontinent       50
country            50
city            15968
dtype: int64

Перевіряємо, чи все коректно замінилося
clean_continent       0
clean_subcontinent    0
clean_country         0
clean_city            0
dtype: int64


### Proper Date/Time Formatting and Timezone Considerations

In [9]:
hits['converted_visitStartTime'] = pd.to_datetime(hits['visitStartTime'], unit='s', utc=True)

print("Sample of converted visitStartTime:", hits['converted_visitStartTime'].head())

Sample of converted visitStartTime: 0   2016-12-24 16:27:48+00:00
1   2016-12-25 06:21:16+00:00
2   2016-12-24 11:05:57+00:00
3   2016-12-24 10:15:51+00:00
4   2016-12-24 18:20:29+00:00
Name: converted_visitStartTime, dtype: datetime64[ns, UTC]


In [10]:
sessions['converted_date'] = pd.to_datetime(sessions['date'])

print("Sample of converted date:", sessions['converted_date'].head())

Sample of converted date: 0   2016-11-19
1   2016-08-18
2   2016-10-20
3   2016-09-12
4   2016-08-10
Name: converted_date, dtype: datetime64[ns]


In [11]:
hits['derived_hour'] = hits['converted_visitStartTime'].dt.hour
hits['derived_minute'] = hits['converted_visitStartTime'].dt.minute
print("Hour mismatches:", (hits['hour'] != hits['derived_hour']).sum())
print("Minute mismatches:", (hits['minute'] != hits['derived_minute']).sum())

# Drop якщо немає помилок
#hits = hits.drop(['derived_hour', 'derived_minute'], axis=1)

Hour mismatches: 125456
Minute mismatches: 81941


### Browser/OS categorization

In [12]:
print("Unique browsers:", sessions['browser'].unique())
print()
print("Unique OS:", sessions['operatingSystem'].unique())

Unique browsers: ['Chrome' 'Safari' 'Firefox' 'Opera Mini' 'Edge' 'UC Browser'
 'Safari (in-app)' 'YaBrowser' 'Opera' 'Android Browser' 'Android Webview'
 'Internet Explorer' 'Maxthon' 'BlackBerry' 'MRCHROME'
 'Mozilla Compatible Agent' 'Nintendo Browser' 'LYF_LS_4002_12'
 'Nokia Browser' 'HTC802t_TD' 'Lunascape' 'Coc Coc' 'Amazon Silk']

Unique OS: ['Android' 'Windows' 'Macintosh' 'iOS' nan 'Linux' 'BlackBerry'
 'Chrome OS' 'Windows Phone' 'Firefox OS' 'Samsung' 'Xbox' 'Nintendo Wii']


In [13]:
top_browsers = sessions["browser"].value_counts().nlargest(5).index
sessions["grouped_browser"] = sessions["browser"].apply(lambda x: x if x in top_browsers else "Other")

top_os = sessions["operatingSystem"].value_counts().nlargest(5).index
sessions["grouped_operatingSystem"] = sessions["operatingSystem"].apply(lambda x: x if x in top_os else "Other")

In [14]:
hits.to_csv("cleaned_data/clean_final_hits_10.csv", index=False)
products.to_csv("cleaned_data/clean_final_products_10.csv", index=False)
sessions.to_csv("cleaned_data/clean_final_sessions_10.csv", index=False)

# Derived Column Creation

## Time-Based Calculations:
### Precise hit timestamps

In [15]:
hits['hitTimestamp'] = hits['converted_visitStartTime'] + pd.to_timedelta(hits['time'], unit='ms')
print(hits["hitTimestamp"].head(5))

0   2016-12-24 16:27:48+00:00
1   2016-12-25 06:21:16+00:00
2   2016-12-24 11:05:57+00:00
3   2016-12-24 10:15:51+00:00
4   2016-12-24 18:20:29+00:00
Name: hitTimestamp, dtype: datetime64[ns, UTC]


### Session duration

In [16]:
session_durations = hits.groupby('uniqueVisitId')['hitTimestamp'].agg(['min', 'max']).reset_index()
session_durations['sessionDurationSec'] = (session_durations['max'] - session_durations['min']).dt.total_seconds()

sessions = sessions.merge(
    session_durations[['uniqueVisitId', 'sessionDurationSec']],
    on='uniqueVisitId',
    how='left'
)

### Time-of-day and day-of-week analysis

In [17]:
hits['time_of_day'] = hits['hitTimestamp'].dt.hour
hits['day_of_week'] = hits['hitTimestamp'].dt.day_name()

sessions['day_of_week'] = sessions['converted_date'].dt.day_name()

In [18]:
def categorize_time_of_day(hour):
    if 0 <= hour < 6:
        return 'Night'
    elif 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    else:
        return 'Evening'
hits['time_of_day_category'] = hits['time_of_day'].apply(categorize_time_of_day)

In [19]:
hits.to_csv("cleaned_data/clean_final_hits_10.csv", index=False)
products.to_csv("cleaned_data/clean_final_products_10.csv", index=False)
sessions.to_csv("cleaned_data/clean_final_sessions_10.csv", index=False)

## Content & Behavior Analysis:
### Page hierarchy extraction

In [20]:
def split_page_path(path):
    if pd.isna(path):
        return [None] * 4
    
    levels = [level for level in path.strip('/').split('/') if level]
    levels += [None] * (4 - len(levels)) if len(levels) < 4 else []
    return pd.Series(levels[:4], index=['pageLevel1', 'pageLevel2', 'pageLevel3', 'pageLevel4'])

page_hierarchy = hits['pagePath'].apply(split_page_path)
hits = pd.concat([hits, page_hierarchy], axis=1)

###  Landing / Exit pages

In [21]:
hits['isLandingPage'] = hits['isEntrance'].astype(str).str.lower().isin(['true', '1'])
hits['isExitPage'] = hits['isExit'].astype(str).str.lower().isin(['true', '1'])

# Визначення першої сторінки сесії
landing_pages = hits[hits['isLandingPage']].groupby('uniqueVisitId')['pagePath'].first().reset_index()
landing_pages.columns = ['uniqueVisitId', 'landingPage']

# Визначення останньої сторінки сесії
exit_pages = hits[hits['isExitPage']].groupby('uniqueVisitId')['pagePath'].last().reset_index()
exit_pages.columns = ['uniqueVisitId', 'exitPage']  # <- Ось тут виправлено

hits = hits.merge(landing_pages, on='uniqueVisitId', how='left').merge(exit_pages, on='uniqueVisitId', how='left')

In [22]:
print(hits.head(5))

                    uniqueVisitId                             hitId  \
0  2133446612777144939:1482596868  2133446612777144939:1482596868:1   
1  1866927528300802411:1482646876  1866927528300802411:1482646876:1   
2  2686045673634323335:1482577557  2686045673634323335:1482577557:1   
3  9491067798164879271:1482574551  9491067798164879271:1482574551:1   
4  5359359794179653383:1482603629  5359359794179653383:1482603629:1   

   visitStartTime  hitNumber  time  hour  minute isEntrance isExit  \
0      1482596868          1     0     8      27       True   True   
1      1482646876          1     0    22      21       True   True   
2      1482577557          1     0     3       5       True   True   
3      1482574551          1     0     2      15       True   True   
4      1482603629          1     0    10      20       True   True   

                                             referer  ... day_of_week  \
0  https://www.baidu.com/link?url=h8Pv4XJCr7Nct1u...  ...    Saturday   
1     

### New vs. Returning

In [23]:
sessions['visitorType'] = sessions['visitnumber'].apply(lambda x: 'New' if x == 1 else 'Returning')

In [24]:
hits.to_csv("cleaned_data/clean_final_hits_10.csv", index=False)
products.to_csv("cleaned_data/clean_final_products_10.csv", index=False)
sessions.to_csv("cleaned_data/clean_final_sessions_10.csv", index=False)

## Marketing & Geographic Insights:
### Marketing channel hierarchies and groupings

In [25]:
sessions['sourceMedium'] = sessions['source'] + ' / ' + sessions['medium']
# Зроблено тільки це, так як вже існує окрема колонка 'channelGrouping'

In [26]:
print(sessions['channelGrouping'].unique())

['Organic Search' 'Social' 'Direct' 'Paid Search' 'Referral' 'Affiliates'
 'Display']


### Device and platform categorizations

In [31]:
hits.to_csv("cleaned_data/clean_final_hits_10.csv", index=False)
products.to_csv("cleaned_data/clean_final_products_10.csv", index=False)
sessions.to_csv("cleaned_data/clean_final_sessions_10.csv", index=False)

In [28]:
sessions

,date,uniqueVisitId,visitnumber,visitid,source,medium,campaign,keyword,adcontent,channelGrouping,...,clean_subcontinent,clean_country,clean_city,converted_date,grouped_browser,grouped_operatingSystem,sessionDurationSec,day_of_week,visitorType,sourceMedium
0,2016-11-19,0267115100035109421:1479582662,1,1479582662,google,organic,NaN,NaN,NaN,Organic Search,...,Northern Europe,Latvia,No data,2016-11-19,Chrome,Android,43.372,Saturday,New,google / organic
1,2016-08-18,0422668257529133405:1471535264,1,1471535264,google,organic,NaN,NaN,NaN,Organic Search,...,Western Asia,Saudi Arabia,Jeddah,2016-08-18,Chrome,Windows,0.000,Thursday,New,google / organic
2,2016-10-20,0472618181924234293:1476976795,1,1476976795,youtube.com,referral,NaN,NaN,NaN,Social,...,Western Asia,United Arab Emirates,No data,2016-10-20,Chrome,Windows,108.566,Thursday,New,youtube.com / referral
3,2016-09-12,0686446117744288372:1473699297,1,1473699297,youtube.com,referral,NaN,NaN,NaN,Social,...,Southern Europe,Serbia,No data,2016-09-12,Safari,Macintosh,0.000,Monday,New,youtube.com / referral
4,2016-08-10,088627669299550779:1470860350,1,1470860350,youtube.com,referral,NaN,NaN,NaN,Social,...,Eastern Europe,Bulgaria,No data,2016-08-10,Safari,Macintosh,0.000,Wednesday,New,youtube.com / referral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27040,2016-11-11,5272378399868786780:1478921989,3,1478921989,youtube.com,referral,NaN,NaN,NaN,Social,...,Southeast Asia,Vietnam,Bien Hoa,2016-11-11,Chrome,Windows,0.000,Friday,Returning,youtube.com / referral
27041,2016-08-22,605117567359942638:1471878456,1,1471878456,youtube.com,referral,NaN,NaN,NaN,Social,...,Southeast Asia,Vietnam,Hanoi,2016-08-22,Chrome,Windows,0.000,Monday,New,youtube.com / referral
27042,2016-11-09,6445859330432245007:1478741098,1,1478741098,youtube.com,referral,NaN,NaN,NaN,Social,...,Southeast Asia,Vietnam,Hanoi,2016-11-09,Chrome,Windows,53.445,Wednesday,New,youtube.com / referral
27043,2016-11-07,7818060723201251292:1478521473,1,1478521473,youtube.com,referral,NaN,NaN,NaN,Social,...,Southeast Asia,Vietnam,Da Nang,2016-11-07,Chrome,Windows,0.000,Monday,New,youtube.com / referral


In [29]:
import pandas as pd



df = products

def fill_category(row, df):
    cat = row["productCategory"]
    
    if pd.notnull(cat) and not cat.startswith("${") and cat.strip() != "":
        return cat.strip()
    
    sku = row["productSKU"]
    name = row["productName"]
    
    sku_matches = df[
        (df["productSKU"] == sku) &
        (df["productCategory"].notnull()) &
        (~df["productCategory"].str.startswith("${", na=False)) &
        (df["productCategory"].str.strip() != "")
    ]
    
    if not sku_matches.empty:
        return sku_matches.iloc[0]["productCategory"].strip()
    
    name_matches = df[
        (df["productName"] == name) &
        (df["productCategory"].notnull()) &
        (~df["productCategory"].str.startswith("${", na=False)) &
        (df["productCategory"].str.strip() != "")
    ]
    
    if not name_matches.empty:
        return name_matches.iloc[0]["productCategory"].strip()
    

    return "Unknown"


df["CleanedCategory"] = df.apply(lambda row: fill_category(row, df), axis=1)

In [30]:
df

,productSKU,productName,productCategory,productVariant,productBrand,productRevenue,productPrice,productQuantity,transactionId,uniqueVisitId,hitId,group_id,productRevenue_converted,productPrice_converted,CleanedCategory
0,GGOEGBRJ037299,Google Alpine Style Backpack,Home/Bags/,NaN,NaN,NaN,99990000,NaN,NaN,7062201778804207621:1492294006,7062201778804207621:1492294006:3,9,NaN,99.99,Home/Bags/
1,GGOEGAAX0343,Google Women's Vintage Hero Tee Lavender,Home/Apparel/Women's/Women's-T-Shirts/,NaN,NaN,NaN,18990000,NaN,NaN,5319403063953973872:1492280129,5319403063953973872:1492280129:3,9,NaN,18.99,Home/Apparel/Women's/Women's-T-Shirts/
2,GGOEGAAX0360,Google Women's Fleece Hoodie,Home/Apparel/Women's/Women's-Outerwear/,NaN,NaN,NaN,55990000,NaN,NaN,9327235454667934962:1492257824,9327235454667934962:1492257824:4,9,NaN,55.99,Home/Apparel/Women's/Women's-Outerwear/
3,GGOEGAAX0360,Google Women's Fleece Hoodie,Home/Apparel/Women's/Women's-Outerwear/,NaN,NaN,NaN,55990000,NaN,NaN,9327235454667934962:1492257824,9327235454667934962:1492257824:5,9,NaN,55.99,Home/Apparel/Women's/Women's-Outerwear/
4,GGOEAKDH019899,Windup Android,Home/Accessories/,NaN,NaN,NaN,3990000,NaN,NaN,5665034960584533055:1492323156,5665034960584533055:1492323156:4,9,NaN,3.99,Home/Accessories/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37713,GGOEGHPB071610,Google Twill Cap,Home/Apparel/Headgear/,NaN,NaN,NaN,10990000,NaN,NaN,2489720925336326856:1477149130,2489720925336326856:1477149130:7,9,NaN,10.99,Home/Apparel/Headgear/
37714,GGOEGAAX0105,Google Men's 100% Cotton Short Sleeve Hero Tee...,Home/Apparel/,NaN,NaN,NaN,16990000,NaN,NaN,1409981326749954901:1477147093,1409981326749954901:1477147093:4,9,NaN,16.99,Home/Apparel/
37715,GGOEGAAX0105,Google Men's 100% Cotton Short Sleeve Hero Tee...,Home/Apparel/,NaN,NaN,NaN,16990000,NaN,NaN,1409981326749954901:1477147093,1409981326749954901:1477147093:6,9,NaN,16.99,Home/Apparel/
37716,GGOEGAAX0105,Google Men's 100% Cotton Short Sleeve Hero Tee...,Home/Apparel/,NaN,NaN,NaN,16990000,NaN,NaN,1409981326749954901:1477147093,1409981326749954901:1477147093:9,9,NaN,16.99,Home/Apparel/
